In [1]:
import torch, os
from pytorch_pretrained_bert import OpenAIGPTTokenizer, OpenAIGPTLMHeadModel
from tqdm import tqdm
import numpy as np
from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE
from pytorch_pretrained_bert.modeling import BertForSequenceClassification, BertConfig, WEIGHTS_NAME, CONFIG_NAME
#from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.optimization import BertAdam, warmup_linear
from bertviz.bertviz.pytorch_pretrained_bert import BertModel, BertTokenizer

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
special_tokens = ['<POS>', '<NEG>','<CON_START>', '<REPLACE>','<START>','<END>'] # Set the special tokens
tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt', special_tokens=special_tokens)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = OpenAIGPTLMHeadModel.from_pretrained('openai-gpt', num_special_tokens=len(special_tokens))

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [3]:
model.config.n_positions

512

In [4]:
bert_classifier_dir = "../models/BERT/" 
model_cls = BertForSequenceClassification.from_pretrained(bert_classifier_dir, num_labels=2)
tokenizer_cls = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
model_cls.to(device)
model_cls.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
    

In [5]:
max_seq_len=70
sm = torch.nn.Softmax(dim=-1)

## Delete, Retrieve and Generate

In [6]:
special_tokens = ['<ATTR_WORDS>','<CON_START>','<REPLACE>','<START>','<END>'] # Set the special tokens
tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt', special_tokens=special_tokens)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = OpenAIGPTLMHeadModel.from_pretrained('openai-gpt', num_special_tokens=len(special_tokens))

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [7]:
model_dir = "/home/jack/Desktop/NN/clean/models/Generator/pytorch_model_zero_grad_1.bin"

model_state_dict = torch.load(model_dir, map_location=device)
model.load_state_dict(model_state_dict)
model.to(device)
model.eval()

OpenAIGPTLMHeadModel(
  (transformer): OpenAIGPTModel(
    (tokens_embed): Embedding(40483, 768)
    (positions_embed): Embedding(512, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_1): BertLayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): BertLayerNorm()
      )
      (1): Block(
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_1): BertLayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout

In [8]:
def preditction_with_beam_search(ref_text, beam_width=3, vocab_length=40484):
    """
    This function decodes sentences using Beam Seach. 
    It will output #sentences = beam_width. This function works on a single example.
    
    ref_text : string : Input sentence
    beam_width : int : Width of the output beam
    vocab_length : int : Size of the Vocab after adding the special tokens
    """
    
    done = [False for i in range(beam_width)] # To track which beams are already decoded
    stop_decode = False
    decoded_sentences=[] # List of decoded sentences at any given time
    
    sm = torch.nn.Softmax(dim=-1) # To calculate Softmax over the final layer Logits
    tokens = tokenizer.tokenize(ref_text) # Tokenize the input text
    
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokens) # Convert tokens to ids
    index_tokens = [indexed_tokens for i in range(beam_width)] # Replication of Input ids for all the beams

    #index_tokens = [indexed_tokens for i in range(beam_width)]
    torch_tensor = torch.tensor(index_tokens).to(device)
    beam_indexes = [[] for i in range(beam_width)] # indexes of the current decoded beams
    best_scoes = [0 for i in range(beam_width)] # A list of lists to store Probability values of each decoded token of best beams
    count = 0
    while count < model.config.n_positions and not stop_decode:
        if count == 0: # For the first step when only one sentence is availabe
            with torch.no_grad():
                # Calculate output probability distribution over the Vocab,
                preds = sm(model(torch_tensor)) #  shape = [beam_bidth, len(input_sen)+1,Vocab_length]
            top_v, top_i = preds[:,-1,:].topk(beam_width) # Fatch top indexes and it's values
            [beam_indexes[i].append(top_i[0][i].tolist()) for i in range(beam_width)] # Update the Beam indexes
            # Update the best_scores, for first time just add the topk values directly
            for i in range(beam_width):
                best_scoes[i] = top_v[0][i].item()
            count += torch_tensor.shape[1]
        else: # After first step
            # Prepare the current_state by concating original input and decoded beam indexes
            current_state = torch.cat((torch_tensor, torch.tensor(beam_indexes).to(device)), dim=1)
            # Prediction on the current state
            with torch.no_grad():
                preds = sm(model(current_state))
            # Multiply new probability predictions with corresponding best scores
            # Total socres = beam_width * Vocab_Size
            flatten_score = (preds[:,-1,:]*torch.tensor(best_scoes).to(device).unsqueeze(1)).view(-1)
            # Fatch the top scores and indexes 
            vals, inx = flatten_score.topk(beam_width)
            # best_score_inx saves the index of best beams after multiplying the probability of new prediction
            best_scoes_inx = (inx / vocab_length).tolist()
            best_scoes = vals.tolist()
            # Unflatten the index 
            correct_inx = (inx % vocab_length).tolist()
            
            # Check if done for all the Beams
            for i in range(beam_width):
                if correct_inx[i] == tokenizer.special_tokens["<END>"]:
                    done[i] = True
            # Update the best score for each the current Beams
            for i in range(beam_width):
                if not done[i]:
                    best_scoes[i] = vals.tolist()[i]
            # Check is All the Beams are Done
            if (sum(done) == beam_width):
                stop_decode = True
            # Prepapre the new beams
            temp_lt=[0 for i in range(beam_width)]
            for i,x in enumerate(best_scoes_inx):
                temp_lt[i] = beam_indexes[i] + [correct_inx[i]]
            # Update the Beam indexes
            beam_indexes = temp_lt
            del temp_lt
            count += 1
    # Decode All the beam indexes to till <END> token only and convert into sentence
    for i in range(beam_width):
        try:
            end_index = beam_indexes[i].index(tokenizer.special_tokens["<END>"])
        except ValueError:
            end_index = len(beam_indexes[i])
            
        decoded_sentences.append(tokenizer.decode(beam_indexes[i][:end_index]))
        
    return decoded_sentences

In [9]:
def get_best_sentence(input_sentences, sentiment=1):
    """
    This function selects the sentence from the Beam of the sentences,
    based on the classification probability score.
    
    input_sentences : list of strings : Sentences generated by the Beam search decoding
    sentiment: int : Expected sentiment (in general class for the classification)
    """
    # BERT pre-processing
    ids = []
    segment_ids = []
    input_masks = []
    pred_lt = []
    for sen in input_sentences:
        text_tokens = tokenizer_cls.tokenize(sen)
        tokens = ["[CLS]"] + text_tokens + ["[SEP]"]
        temp_ids = tokenizer_cls.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(temp_ids)
        segment_id = [0] * len(temp_ids)
        padding = [0] * (max_seq_len - len(temp_ids))

        temp_ids += padding
        input_mask += padding
        segment_id += padding
        
        ids.append(temp_ids)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
    ids = torch.tensor(ids).to(device)
    segment_ids = torch.tensor(segment_ids).to(device)
    input_masks = torch.tensor(input_masks).to(device)
    # prediction
    with torch.no_grad():
        preds = sm(model_cls(ids, segment_ids, input_masks))
        
    preds = preds.tolist()
    inx, inx_val = None, 0
    for i in range(len(input_sentences)):
        temp = preds[i][sentiment]
        if temp > inx_val:
            inx = i
            inx_val = temp
    return input_sentences[inx]

## Example

In [10]:
op=preditction_with_beam_search("<ATTR_WORDS> good very <CON_START> for the price , this place is <REPLACE> <REPLACE> . <START> ",beam_width=5, vocab_length=max(tokenizer.special_tokens.values()) + 1)
op

['for the price , this place is very good .',
 '$ for prices , this place is very very good .',
 'to the dollar , this place is very , very good $ for place .',
 'as this fee , this place is very good .',
 'money for good , this place is good good .']

In [11]:
print(get_best_sentence(op, sentiment=1))

for the price , this place is very good .


In [12]:
# Output dir have reference files generated using TFIDF for retrieve attributes from opposite corpus
data_dir = "/home/jack/Desktop/NN/clean/datasets/yelp"
output_dir = data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/tfidf/"

os.listdir(output_dir)

['reference_1', 'reference_0', 'backup']

In [15]:
with open(os.path.join(output_dir,"reference_0_predictions_with_full_sentence_match_beam_search_bm5.txt") ,"w", encoding='utf-8') as out_fp:
    c = 0
    with open(os.path.join(output_dir, "reference_0.txt")) as fp:
        for line in fp:
            out_sen = preditction_with_beam_search(line.strip(), beam_width=5, vocab_length=max(tokenizer.special_tokens.values()) + 1)
            print(c,get_best_sentence(out_sen, sentiment = 1))
            c += 1
            out_fp.write(get_best_sentence(out_sen, sentiment = 1) + "\n")

0 always since joes has changed hands it ' s just just fast and friendly .
1 it there no does not enough room in that part does well .
2 and so works working down .
3 i honestly she ' d be back and for a few talent .
4 ( do n ' t believe how inconsiderate this great rest is . i i ca n ' t believe how inconsiderate this great . )
5 its slide and cool it off the bill .
6 ' it is n ' big , but it is n ' t too bad either .
7 loved love that i could not use my birthday gift !
8 fresh owner , i feel amazing - but i do n ' t know the details .
9 but it probably town great !
10 ( sit down and we got some really fun and continue service . . . we we definitely we definitely continue
11 the lovely did include miso soup and a lovely salad .
12 ( really food i ' m delicious delicious or how delicious did out .
13 maybe we could n ' afford wait eat at the asset if we were n ' t ordering dinner . we could n ' t definitely afford afford wait at the asset if we were n ' t ordering dinner but we
14 the 

In [16]:
with open(os.path.join(output_dir,"reference_1_predictions_with_full_sentence_match_beam_search_bm5.txt") ,"w", encoding='utf-8') as out_fp:
    c = 0
    with open(os.path.join(output_dir, "reference_1.txt")) as fp:
        for line in fp:
            out_sen = preditction_with_beam_search(line.strip(), beam_width=5, vocab_length=max(tokenizer.special_tokens.values()) + 1)
            c += 1
            out_fp.write(get_best_sentence(out_sen, sentiment = 0) + "\n")

## Delete and Generate

In [ ]:
path = os.path.join(os.getcwd(), "./log_yelp_bert_best_head_preprocessed/pytorch_model_zero_grad_1.bin") ## Model Path
model_state_dict = torch.load(path, map_location=device)
model.load_state_dict(model_state_dict)
model.to(device)
model.eval()

In [ ]:
output_dir = "/home/ubuntu/bhargav/data/yelp/processed_files_with_bert_with_best_head/"
os.listdir(output_dir)

### Predictions for the reference files

In [ ]:
with open(os.path.join(output_dir,"reference_0_predictions_with_beam_search.txt") ,"w", encoding='utf-8') as out_fp:
    c = 0
    with open(os.path.join(output_dir, "./reference_0.txt")) as fp:
        for line in fp:
            out_sen = preditction_with_beam_search(line.strip(), beam_width=5, vocab_length=max(tokenizer.special_tokens.values()) + 1)
            print(c,get_best_sentence(out_sen, sentiment = 1))
            c += 1
            out_fp.write(get_best_sentence(out_sen, sentiment = 1) + "\n")

In [ ]:
with open(os.path.join(output_dir,"reference_1_predictions_with_beam_search.txt") ,"w", encoding='utf-8') as out_fp:
    c = 0
    with open(os.path.join(output_dir, "./reference_1.txt")) as fp:
        for line in fp:
            out_sen = preditction_with_beam_search(line.strip(), beam_width=5, max(tokenizer.special_tokens.values()) + 1)
            print(c,get_best_sentence(out_sen, sentiment = 0))
            c += 1
            out_fp.write(get_best_sentence(out_sen, sentiment = 0) + "\n")